In [1]:
import time
import numpy as np
import math
import random

### Algorithm: Simulated Annealing 
<b> Idea of metallurgy in AI
    1. In high temperature it accept the best move as well as worst move that is above certain probability.
    2. As temperature decrease gradually it choose only the best move.
    
    ** As temperature decrease metals become stronger (atomic bond) and similarly in AI the next move become better and best.
    
### Problem: N Queen

### Pseudocode
1. Let s = s0
2. For k = 0 through kmax (exclusive):
3.     * T ← temperature( (k+1)/kmax )
4.     * Pick a random neighbour, snew ← neighbour(s)
5.     * If P(E(s), E(snew), T) ≥ random(0, 1):
6.     * s ← snew
7. Output: the final state s

#### Heuristic: Number of pair in attack situation.

In [2]:
'''
Each row contains one queen in arbitrary column.  
So, Vartical and diagonal attack only. No horizontal attack.
'''
def calc_attack_heuristic(state):
    attack=0
    for row, col in enumerate(state): 
        for ir in range(row+1, len(state) ): 
    #         print('row=',row,'q col=',col,' ir=',ir, 'current[ir]=', current[ir])
            if state[ir]==col:   #vertical attack
                attack+=1 
            elif abs(col-state[ir])==ir-row :  #diagonal attack
                attack+=1
    return -attack

In [3]:
def board_print(state):
    bs=''
    for row in range( len(state) ):
        for col in range( len(state) ):
            if col==state[row]:
                bs+='Q '
            else:
                bs+='. '
        if row!=len(state)-1:
            bs+='\n'
    print(state)
    print('')
    print(bs)

In [4]:
'''
For each of the n rows queen can move to n-1 other positions. So, #next_states= n*(n-1)
'''
def gen_next_states(state):
    near_states=[]
    for row in range( len(state) ):
        for col in range( len(state) ):
            if col==state[row]:
                pass
            else:
                near = list(state)   #copy
                near[row]=col        #place queen in new position for this row.
                near_states.append(near)
    return near_states

In [5]:
state=np.random.randint(low=0, high=3, size=4)
board_print(state)

[0 2 2 2]

Q . . . 
. . Q . 
. . Q . 
. . Q . 


In [6]:
near_states=gen_next_states(state)
print(state)
for ns in near_states:
    print(ns, calc_attack_heuristic(ns))

[0 2 2 2]
([1, 2, 2, 2], -4)
([2, 2, 2, 2], -6)
([3, 2, 2, 2], -4)
([0, 0, 2, 2], -4)
([0, 1, 2, 2], -4)
([0, 3, 2, 2], -3)
([0, 2, 0, 2], -2)
([0, 2, 1, 2], -3)
([0, 2, 3, 2], -3)
([0, 2, 2, 0], -4)
([0, 2, 2, 1], -3)
([0, 2, 2, 3], -4)


In [7]:
def is_it_goal(state):
    if calc_attack_heuristic(state)==0:
        return True
    else:
        return False

In [71]:
def cooling_schedule(t, mx): 
    fraction=(t+1)/float(mx)
    return max(0.01,  min(1, 1 - fraction))

In [72]:
def sim_annealing(nqueen):
    current=np.random.randint(low=0, high=nqueen-1, size=nqueen)
    mx=1000000 
    for t in range(mx): 
        T=cooling_schedule(t, mx)
        if T == 0 or is_it_goal(current):
                break
        ec=calc_attack_heuristic(current)
        next_states=gen_next_states(current)
        state = next_states[np.random.randint(len(next_states))]
        en=calc_attack_heuristic(state)
        delta=en-ec
        if delta>0:
            current=state
        elif np.exp( delta/T )>np.random.uniform(0.0, 1.0):
            current=state
    return current

In [73]:
nqueen=10
print('#queen %s'%nqueen)
st=time.time()
current=sim_annealing(nqueen)
et=time.time()

print('\nduration: %0.2f seconds'%(et-st))
success=is_it_goal(current) 
print('goal found=%s'% success )
if success: 
    board_print(current)

#queen 10

duration: 1.39 seconds
goal found=True
[7, 1, 4, 2, 8, 6, 9, 3, 5, 0]

. . . . . . . Q . . 
. Q . . . . . . . . 
. . . . Q . . . . . 
. . Q . . . . . . . 
. . . . . . . . Q . 
. . . . . . Q . . . 
. . . . . . . . . Q 
. . . Q . . . . . . 
. . . . . Q . . . . 
Q . . . . . . . . . 
